In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

## Data Collection and Preprocessing

In [2]:
df = pd.read_csv(r"C:\Users\ASUS\OneDrive\Desktop\LTTS\New folder\medipin website\medical_website-main\drugs_side_effects_drugs_com.csv.xls")

In [3]:
df=df.iloc[:,:17]
df

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html
2,minocycline,Acne,"skin rash, fever, swollen glands, flu-like sym...",minocycline,Tetracyclines,"Dynacin, Minocin, Minolira, Solodyn, Ximino, V...",48%,Rx,D,N,NaN,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,5.7,482,https://www.drugs.com/minocycline.html,https://www.drugs.com/condition/acne.html
3,Accutane,Acne,problems with your vision or hearing; muscle o...,isotretinoin (oral),"Miscellaneous antineoplastics, Miscellaneous u...",NaN,41%,Rx,X,N,X,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.9,623,https://www.drugs.com/accutane.html,https://www.drugs.com/condition/acne.html
4,clindamycin,Acne,hives ; difficult breathing; swelling of your ...,clindamycin topical,"Topical acne agents, Vaginal anti-infectives","Cleocin T, Clindacin ETZ, Clindacin P, Clindag...",39%,Rx,B,N,NaN,doxycycline: https://www.drugs.com/doxycycline...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.4,146,https://www.drugs.com/mtm/clindamycin-topical....,https://www.drugs.com/condition/acne.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2961,benzphetamine,Weight Loss,hives ; difficulty breathing; swelling of your...,benzphetamine,"Anorexiants, CNS stimulants","Didrex, Regimex",2%,Rx,X,3,X,phentermine: https://www.drugs.com/phentermine...,Weight Loss (Obesity/Overweight) More than hal...,7.5,17,https://www.drugs.com/mtm/benzphetamine.html,https://www.drugs.com/condition/obesity.html
2962,fenfluramine,Weight Loss,hives ; difficult breathing; swelling of your ...,fenfluramine,"Anorexiants, CNS stimulants, Miscellaneous ant...",Fintepla,1%,Rx,C,4,X,NaN,Weight Loss (Obesity/Overweight) More than hal...,NaN,NaN,https://www.drugs.com/mtm/fenfluramine.html,https://www.drugs.com/condition/obesity.html
2963,Fendique ER,Weight Loss,NaN,phendimetrazine tartrate,"Anorexiants, CNS stimulants",NaN,0%,Rx,C,3,X,phentermine: https://www.drugs.com/phentermine...,Weight Loss (Obesity/Overweight) More than hal...,NaN,NaN,https://www.drugs.com/pro/fendique-er.html,https://www.drugs.com/condition/obesity.html
2964,Imcivree,Weight Loss,"Imcivree may cause serious side effects, inclu...",setmelanotide,Melanocortin receptor agonists,NaN,0%,Rx,NaN,N,NaN,phentermine: https://www.drugs.com/phentermine...,Weight Loss (Obesity/Overweight) More than hal...,NaN,NaN,https://www.drugs.com/imcivree.html,https://www.drugs.com/condition/obesity.html


In [4]:
df.shape

(2966, 17)

In [5]:
#import pandas_profiling
# checking for misssing values
#df.isnull().sum()

In [6]:
df = df.drop(columns=['drug_link','medical_condition_url','related_drugs','brand_names','drug_classes','rx_otc','csa','pregnancy_category','side_effects','medical_condition_description'],axis=1)
df

,drug_name,medical_condition,generic_name,activity,alcohol,rating,no_of_reviews
0,doxycycline,Acne,doxycycline,87%,X,6.8,760
1,spironolactone,Acne,spironolactone,82%,X,7.2,449
2,minocycline,Acne,minocycline,48%,NaN,5.7,482
3,Accutane,Acne,isotretinoin (oral),41%,X,7.9,623
4,clindamycin,Acne,clindamycin topical,39%,NaN,7.4,146
...,...,...,...,...,...,...,...
2961,benzphetamine,Weight Loss,benzphetamine,2%,X,7.5,17
2962,fenfluramine,Weight Loss,fenfluramine,1%,X,NaN,NaN
2963,Fendique ER,Weight Loss,phendimetrazine tartrate,0%,X,NaN,NaN
2964,Imcivree,Weight Loss,setmelanotide,0%,NaN,NaN,NaN


Alcohol

In [7]:
plt.show()

In [8]:
df['alcohol'].value_counts()

X                                    1377
2021                                   34
N=""function""==typeof Uint8Array       1
Name: alcohol, dtype: int64

In [9]:
df['alcohol'].fillna('2021',inplace = True)

In [10]:
df.isna().sum()

drug_name               1
medical_condition       0
generic_name           44
activity                1
alcohol                 0
rating               1361
no_of_reviews        1361
dtype: int64

In [11]:
y=0
df['alcohol'].value_counts()
for i in df['alcohol']:
  if i != '2021' and i!='X':
    print(y)
  y+=1

2008


In [12]:
df.drop([2008],axis=0,inplace=True)

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['alcohol'] = le.fit_transform(df['alcohol'])

In [14]:
df['alcohol']

0       1
1       1
2       0
3       1
4       0
       ..
2961    1
2962    1
2963    1
2964    0
2965    0
Name: alcohol, Length: 2965, dtype: int32

### Activity

In [15]:
df['activity'].unique()

array(['87%', '82%', '48%', '41%', '39%', '35%', '30%', '26%', '20%',
       '17%', '13%', '12%', '9%', '8%', '7%', '6%', '5%', '4%', '3%',
       '2%', '1%', '0%', '100%', '95%', '51%', '40%', '38%', '36%', '32%',
       '31%', '27%', '24%', '23%', '16%', '14%', '10%', '97%', '80%',
       '71%', '61%', '60%', '57%', '53%', '50%', '37%', '34%', '33%',
       '18%', '11%', '83%', '75%', '74%', '54%', '98%', '85%', '84%',
       '64%', '19%', '79%', '67%', '42%', '15%', '94%', '78%', '59%',
       '55%', '28%', '92%', '88%', '49%', '22%', '96%', '63%', '45%',
       '43%', '46%', '25%', '21%', '44%', '47%', '29%', '66%',
       ' placebo-controlled Phase2/3 SPECTRA Trial had met its primary and secondary endpoints. Efficacy against severe COVID-19 and hospitalization was 100%',
       '99%', '68%', '73%', '72%', '65%', '52%', nan, '58%', '86%', '62%',
       '91%', '90%'], dtype=object)

In [16]:
y=0
df['activity'].value_counts()
for i in df['activity']:
  if i == ' placebo-controlled Phase2/3 SPECTRA Trial had met its primary and secondary endpoints. Efficacy against severe COVID-19 and hospitalization was 100%':
    print(y)
  y+=1

1116
1118
1120
1122
1124
1126
1128
1130
1132
1134
1136
1138
1140
1142
1144
1146
1148
1150
1152
1154
1156
1158
1160
1162
1164
1166
1168
1170
1172
1174
1176
1178
1180
1182


In [17]:
df['activity'] = df['activity'].replace(' placebo-controlled Phase2/3 SPECTRA Trial had met its primary and secondary endpoints. Efficacy against severe COVID-19 and hospitalization was 100%',100)

In [18]:
df["activity"] = df["activity"].astype(str)

In [19]:
y=0
df['activity'].value_counts()
for i in df['activity']:
  if i == 'h=function(n){document.body?k(document.body):0<n?l.setTimeout(function(){h(n-1)}':
    print(y)
  y+=1

In [20]:
replace_dict = {'87%': 87,'82%': 82,'48%':48,'41%':41,'39%':39,'35%':35,'30%':30,'26%':26,'20%':20,'17%':17,'13%':13,'12%':12,'9%':9,'8%':8,'7%':7,'6%':6,'5%':5,'4%':4,'3%':3,'2%':2,'1%':1,'0%':0,'100%':100,'95%':95,'51%':51,'40%':40,'38%':38,'36%':36,'32%':32,'31%':31,'27%':27,'24%':24,'23%':23,'16%':16,'14%':14,'10%':10,'97%':97,'80%':80,'71%':71,'61%':61,'60%':60,'57%':57,'53%':53,'50%':50,'37%':37,'34%':34,'33%':33,'18%':18,'11%':11,'83%':83,'75%':75,'74%':74,'54%':54,'98%':98,'85%':85,'84%':84,'64%':64,'19%':19,'79%':79,'67%':67,'42%':42,'15%':15,'94%':94,'78%':78,'59%':59,'55%':55,'28%':28,'92%':92,'88%':88,'49%':49,'22%':22,'96%':96,'63%':63,'45%':45,'43%':43,'46%':46,'25%':25,'21%':21,'44%':44,'47%':47,'29%':29,'66%':66,'99%':99,'68%':68,'73%':73,'72%':72,'65%':65,'52%':52,'58%':58,'86%':86,'62%':62,'91%':91,'90%':90,'h=function(n){document.body?k(document.body):0<n?l.setTimeout(function(){h(n-1)}':0,}
df["activity"] = df["activity"].replace(replace_dict) 
df['activity'].unique()

array([87, 82, 48, 41, 39, 35, 30, 26, 20, 17, 13, 12, 9, 8, 7, 6, 5, 4,
       3, 2, 1, 0, 100, 95, 51, 40, 38, 36, 32, 31, 27, 24, 23, 16, 14,
       10, 97, 80, 71, 61, 60, 57, 53, 50, 37, 34, 33, 18, 11, 83, 75, 74,
       54, 98, 85, 84, 64, 19, 79, 67, 42, 15, 94, 78, 59, 55, 28, 92, 88,
       49, 22, 96, 63, 45, 43, 46, 25, 21, 44, 47, 29, 66, '100', 99, 68,
       73, 72, 65, 52, 'nan', 58, 86, 62, 91, 90], dtype=object)

In [21]:
df['activity'].isnull().sum()

0

In [22]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(df['activity'].values.reshape(-1,1))
df['activity'] = imputer.transform(df['activity'].values.reshape(-1,1))

In [23]:
df['activity'].unique()

array([ 87.        ,  82.        ,  48.        ,  41.        ,
        39.        ,  35.        ,  30.        ,  26.        ,
        20.        ,  17.        ,  13.        ,  12.        ,
         9.        ,   8.        ,   7.        ,   6.        ,
         5.        ,   4.        ,   3.        ,   2.        ,
         1.        ,   0.        , 100.        ,  95.        ,
        51.        ,  40.        ,  38.        ,  36.        ,
        32.        ,  31.        ,  27.        ,  24.        ,
        23.        ,  16.        ,  14.        ,  10.        ,
        97.        ,  80.        ,  71.        ,  61.        ,
        60.        ,  57.        ,  53.        ,  50.        ,
        37.        ,  34.        ,  33.        ,  18.        ,
        11.        ,  83.        ,  75.        ,  74.        ,
        54.        ,  98.        ,  85.        ,  84.        ,
        64.        ,  19.        ,  79.        ,  67.        ,
        42.        ,  15.        ,  94.        ,  78.  

In [24]:
df.isna().sum()

drug_name               0
medical_condition       0
generic_name           44
activity                0
alcohol                 0
rating               1361
no_of_reviews        1361
dtype: int64

### No. Of reviews

In [25]:
df['no_of_reviews'].unique()

array(['760', '449', '482', '623', '146', '8', '439', '999', '96', '86',
       '227', '51', '329', '698', '499', '57', '55', '2', '99', '30', '4',
       '570', '5', '193', '94', '32', '1', nan, '116', '13', '267', '909',
       '76', '27', '35', '252', '274', '115', '10', '33', '3', '9', '54',
       '34', '183', '669', '489', '45', '25', '6', '7', '36', '52', '149',
       '42', '11', '37', '187', '46', '12', '28', '768', '360', '207',
       '382', '799', '73', '21', '833', '478', '93', '894', '160', '194',
       '133', '61', '152', '257', '127', '558', '106', '119', '17', '18',
       '14', '266', '41', '24', '124', '53', '26', '171', '122', '89',
       '103', '19', '120', '58', '107', '60', '213', '181', '64', '66',
       '15', '81', '552', '774', '840', '654', '1034', '459', '330',
       '899', '520', '1471', '450', '295', '233', '128', '381', '264',
       '402', '501', '366', '67', '44', '84', '39', '195', '65', '43',
       '600', '634', '383', '332', '328', '334', '681',

In [26]:
y=0
df['no_of_reviews'].value_counts()
for i in df['no_of_reviews']:
  if i == ' double-blinded':
    print(y)
  y+=1

1116
1118
1120
1122
1124
1126
1128
1130
1132
1134
1136
1138
1140
1142
1144
1146
1148
1150
1152
1154
1156
1158
1160
1162
1164
1166
1168
1170
1172
1174
1176
1178
1180
1182


In [27]:
df['no_of_reviews'] = df['no_of_reviews'].replace(' double-blinded',None)
df['no_of_reviews']

0       760
1       449
2       482
3       623
4       146
       ... 
2961     17
2962    NaN
2963    NaN
2964    NaN
2965    NaN
Name: no_of_reviews, Length: 2965, dtype: object

In [28]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(df['no_of_reviews'].values.reshape(-1,1))
df['no_of_reviews'] = imputer.transform(df['no_of_reviews'].values.reshape(-1,1))

In [29]:
df['no_of_reviews']

0       760.000000
1       449.000000
2       482.000000
3       623.000000
4       146.000000
           ...    
2961     17.000000
2962     75.376433
2963     75.376433
2964     75.376433
2965     75.376433
Name: no_of_reviews, Length: 2965, dtype: float64

### Rating

In [30]:
df['rating'].unique()

array(['6.8', '7.2', '5.7', '7.9', '7.4', '7.6', '7.7', '8', '8.5', '7.8',
       '6', '7.3', '6.3', '8.4', '7.5', '10', '6.9', '8.3', '9', '8.2',
       '5', '8.8', nan, '6.7', '7', '6.4', '6.6', '8.7', '8.6', '8.1',
       '4.8', '5.4', '9.5', '5.2', '2', '0', '3.8', '7.1', '8.9', '2.6',
       '4.4', '6.5', '3.7', '6.1', '9.4', '5.8', '5.1', '5.5', '4.5',
       '9.1', '1', '5.9', '4', '9.3', '4.7', '4.6', '4.9', '3.3', '3.9',
       '9.7', '6.2', '2.1', '9.8', '5.3', '5.6', '2.5', '4.2', '3.6',
       '4.3', '3', '9.2', '2.9', '2.8',
       ' with AstraZeneca responsible for development and worldwide manufacturing and distribution. A randomised',
       '9.6', '3.2', '1.8', '2.3', '3.1', '3.4', '3.5', '4.1', '1.7',
       '2.7', '1.5', '1.1', '9.9', '2.4'], dtype=object)

In [31]:
y=0
df['rating'].value_counts()
for i in df['rating']:
  if i == ' with AstraZeneca responsible for development and worldwide manufacturing and distribution. A randomised':
    print(y)
  y+=1

1116
1118
1120
1122
1124
1126
1128
1130
1132
1134
1136
1138
1140
1142
1144
1146
1148
1150
1152
1154
1156
1158
1160
1162
1164
1166
1168
1170
1172
1174
1176
1178
1180
1182


In [32]:
df['rating'] = df['rating'].replace(' with AstraZeneca responsible for development and worldwide manufacturing and distribution. A randomised',None)
df['rating']

0       6.8
1       7.2
2       5.7
3       7.9
4       7.4
       ... 
2961    7.5
2962    NaN
2963    NaN
2964    NaN
2965    NaN
Name: rating, Length: 2965, dtype: object

In [33]:
df['rating'].unique()

array(['6.8', '7.2', '5.7', '7.9', '7.4', '7.6', '7.7', '8', '8.5', '7.8',
       '6', '7.3', '6.3', '8.4', '7.5', '10', '6.9', '8.3', '9', '8.2',
       '5', '8.8', nan, '6.7', '7', '6.4', '6.6', '8.7', '8.6', '8.1',
       '4.8', '5.4', '9.5', '5.2', '2', '0', '3.8', '7.1', '8.9', '2.6',
       '4.4', '6.5', '3.7', '6.1', '9.4', '5.8', '5.1', '5.5', '4.5',
       '9.1', '1', '5.9', '4', '9.3', '4.7', '4.6', '4.9', '3.3', '3.9',
       '9.7', '6.2', '2.1', '9.8', '5.3', '5.6', '2.5', '4.2', '3.6',
       '4.3', '3', '9.2', '2.9', '2.8', None, '9.6', '3.2', '1.8', '2.3',
       '3.1', '3.4', '3.5', '4.1', '1.7', '2.7', '1.5', '1.1', '9.9',
       '2.4'], dtype=object)

In [34]:
df['rating'] = df['rating'].values.reshape(-1,1)

In [35]:
df['rating'].shape

(2965,)

In [36]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(df['rating'].values.reshape(-1,1))
df['rating'] = imputer.transform(df['rating'].values.reshape(-1,1))

In [37]:
df['rating'].isnull().sum()

0

Medical_condition

In [38]:
#df.dropna(subset=['medical_condition'], inplace=True)

In [39]:
#one_hot = pd.get_dummies(df['medical_condition'])
#df = pd.concat([df, one_hot], axis=1)
#print(df['medical_condition'])

In [40]:
#df.drop('medical_condition', axis=1,inplace=True)

In [41]:
#from sklearn.compose import ColumnTransformer
#from sklearn.preprocessing import OneHotEncoder
#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
#df['medical_condition'] = pd.get_dummies(df['medical_condition'])
#df['medical_condition'] = np.array(ct.fit_transform(df['medical_condition']))

In [42]:
# df['medical_condition'] = np.array(df['medical_condition'])
# df['medical_condition']

In [43]:
df['drug_name'].unique()

array(['doxycycline', 'spironolactone', 'minocycline', ..., 'Fendique ER',
       'Imcivree', 'setmelanotide'], dtype=object)

Drug_name

In [44]:
df['generic_name'].fillna(' ',inplace = True)
df['generic_name'].isnull().sum()

0

In [45]:
df['drug_name'].fillna(' ',inplace = True)
df['drug_name'].isnull().sum()

0

In [46]:
df['Drug'] = df['generic_name'] + " "+ df['drug_name']
df['Drug'].isnull().sum()

0

In [47]:
df['Drug'].unique()

array(['doxycycline doxycycline', 'spironolactone spironolactone',
       'minocycline minocycline', ...,
       'phendimetrazine tartrate Fendique ER', 'setmelanotide Imcivree',
       'setmelanotide setmelanotide'], dtype=object)

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2965 entries, 0 to 2965
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   drug_name          2965 non-null   object 
 1   medical_condition  2965 non-null   object 
 2   generic_name       2965 non-null   object 
 3   activity           2965 non-null   float64
 4   alcohol            2965 non-null   int32  
 5   rating             2965 non-null   float64
 6   no_of_reviews      2965 non-null   float64
 7   Drug               2965 non-null   object 
dtypes: float64(3), int32(1), object(4)
memory usage: 196.9+ KB


In [49]:
df.drop(['drug_name', 'generic_name'], axis=1, inplace=True)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2965 entries, 0 to 2965
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   medical_condition  2965 non-null   object 
 1   activity           2965 non-null   float64
 2   alcohol            2965 non-null   int32  
 3   rating             2965 non-null   float64
 4   no_of_reviews      2965 non-null   float64
 5   Drug               2965 non-null   object 
dtypes: float64(3), int32(1), object(2)
memory usage: 150.6+ KB


In [51]:
df['Drug'].shape

(2965,)

In [52]:
y = df['Drug']

In [53]:
df.drop('Drug',axis =1, inplace = True )

In [54]:
for label, content in df.items():
    if pd.api.types.is_numeric_dtype(content):
      if pd.isnull(content).sum():
        df[label]= content.fillna(content.mean())

In [55]:
num_labels=[]

In [56]:
for label, content in df.items():
    if pd.api.types.is_numeric_dtype(content):
      num_labels.append(label)

In [57]:
others = list(df.columns)

In [58]:
for i in num_labels:
  others.remove(i)

In [59]:
df.isnull().sum()

medical_condition    0
activity             0
alcohol              0
rating               0
no_of_reviews        0
dtype: int64

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2965 entries, 0 to 2965
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   medical_condition  2965 non-null   object 
 1   activity           2965 non-null   float64
 2   alcohol            2965 non-null   int32  
 3   rating             2965 non-null   float64
 4   no_of_reviews      2965 non-null   float64
dtypes: float64(3), int32(1), object(1)
memory usage: 127.4+ KB


In [61]:
#df.drop(['medical_condition'], axis=1, inplace=True)

In [62]:
y

0                    doxycycline doxycycline
1              spironolactone spironolactone
2                    minocycline minocycline
3               isotretinoin (oral) Accutane
4            clindamycin topical clindamycin
                        ...                 
2961             benzphetamine benzphetamine
2962               fenfluramine fenfluramine
2963    phendimetrazine tartrate Fendique ER
2964                  setmelanotide Imcivree
2965             setmelanotide setmelanotide
Name: Drug, Length: 2965, dtype: object

In [63]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
ct = make_column_transformer(
    (MinMaxScaler(), num_labels),
    (OneHotEncoder(handle_unknown='ignore'),others))

In [64]:
df.isnull().sum()

medical_condition    0
activity             0
alcohol              0
rating               0
no_of_reviews        0
dtype: int64

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = 0.2, random_state = 1)

In [66]:
X_train.columns

Index(['medical_condition', 'activity', 'alcohol', 'rating', 'no_of_reviews'], dtype='object')

In [67]:
df.columns

Index(['medical_condition', 'activity', 'alcohol', 'rating', 'no_of_reviews'], dtype='object')

In [68]:
ct.fit(X_train)

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 ['activity', 'alcohol', 'rating',
                                  'no_of_reviews']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['medical_condition'])])

In [69]:
x_train = ct.transform(X_train)

In [70]:
x_test = ct.transform(X_test)

In [71]:
x_train

<2372x52 sparse matrix of type '<class 'numpy.float64'>'
	with 9647 stored elements in Compressed Sparse Row format>

In [72]:
y_train

2407             ibuprofen (oral route) Children's Motrin
557                       trifluoperazine trifluoperazine
1039                      magnesium oxide magnesium oxide
561                                 prednisone prednisone
2365                 acetaminophen and oxycodone Percocet
                              ...                        
2764            infliximab-qbtx (intravenous route) Ixifi
905     Dexbrompheniramine and Acetaminophen Caplets D...
1096    budesonide and formoterol (inhalation) budeson...
235     benzoyl peroxide and clindamycin topical Z-Cli...
1061                            Magnesium Oxide Mag-Oxide
Name: Drug, Length: 2372, dtype: object

## Random Forest Classification

In [73]:
# Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 42,max_features='log2')
classifier.fit(x_train, y_train)
y_train1=classifier.predict(x_train)
accuracy_score(y_train, y_train1)

# # Making the Confusion Matrix
# y_pred = classifier.predict(x_test)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)
# accuracy_score(y_test, y_pred)

0.6235244519392917

# Logistic Classification

In [74]:
# Training the Random Forest Classification model on the Training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
y_train1=classifier.predict(x_train)
accuracy_score(y_train,y_train1)

0.050590219224283306